In [1]:

import sys 
sys.path.append('../../')
import sys
sys.path.append('../../../')
sys.path.append('../../')
sys.path.append('../')

from core.conf_defaults import *
from core.tbal import * 
from core.auto_labeling import *
from core.run_lib import * 
from datasets.dataset_utils import * 
from utils.counting_utils import *  
from utils.common_utils import * 
from utils.vis_utils import *
from utils.logging_utils import * 
from  datasets.data_manager import * 
from omegaconf import OmegaConf


root_dir = '../../'
conf_dir = f'{root_dir}configs/tiny-imagenet-CLIP/'

base_conf_file = '{}/tiny_imagenet_CLIP_base_conf.yaml'.format(conf_dir)

logger      = get_logger(f'{root_dir}/temp/logs/pl_tiny_imagenet_CLIP_test.log',stdout_redirect=True,level=logging.DEBUG)

conf = OmegaConf.load(base_conf_file)


set_defaults(conf)

conf['train_pts_query_conf']['max_num_train_pts'] = 10000

conf['data_conf']['compute_emb'] = False


conf['training_conf']['learning_rate']=0.1
conf['training_conf']['optimizer'] = 'sgd'
conf['training_conf']['batch_size']=64
conf['training_conf']['max_epochs']=50
conf['training_conf']['weight_decay']=0.001


In [2]:

set_seed(conf['random_seed'])

dm = DataManager(conf,logger)
len(dm.ds_std_train), len(dm.ds_std_val)


[01/02/2024 08:05:19 PM : INFO  : data_manag : ] : Loaded dataset tiny_imagenet_200_CLIP
[01/02/2024 08:05:19 PM : INFO  : data_manag : ] : Std train size: 100000 and Std. Val. Size:10000


(100000, 10000)

In [3]:
pl = PassiveLearning(conf,dm,logger)

out = pl.run()

[01/02/2024 08:05:20 PM : DEBUG : passive_le : ] : Querying 1000 seed training points
[01/02/2024 08:05:20 PM : DEBUG : passive_le : ] : Queried 1000 seed points for training
[01/02/2024 08:05:21 PM : DEBUG : passive_le : ] : Validation Data Size :10000
[01/02/2024 08:05:21 PM : DEBUG : passive_le : ] : Querying rest of the training points 9000 in single batch
[01/02/2024 08:05:21 PM : INFO  : passive_le : ] : Labeled data size for training: 10000
[01/02/2024 08:05:21 PM : INFO  : pytorch_cl : ] : {'model_name': 'mlp', 'input_dimension': 512, 'lst_dims': [2000, 1000, 500], 'num_classes': 200, 'fit_intercept': True, 'lib': 'pytorch'}
[01/02/2024 08:05:21 PM : INFO  : passive_le : ] : --------------- Begin Model Training ------------
[01/02/2024 08:05:21 PM : INFO  : passive_le : ] : Training conf :{'optimizer': 'sgd', 'learning_rate': 0.1, 'momentum': 0.9, 'batch_size': 64, 'weight_decay': 0.001, 'loss_tolerance': 1e-06, 'max_epochs': 50, 'normalize_weights': False, 'train_err_tol': -1,

In [4]:
conf['auto_lbl_conf']['C_1'] = 0.0
conf['auto_lbl_conf']['auto_label_err_threshold']=0.10
meta_df_cp = copy.deepcopy(dm.meta_df)
auto_labeler = AutoLabeling(conf,dm,pl.cur_clf,logger)
out = auto_labeler.run()
dm.get_auto_labeling_counts()

[01/02/2024 08:05:41 PM : INFO  : auto_label : ] : ========================= Begin Auto-Labeling selective ==========================
[01/02/2024 08:05:41 PM : DEBUG : auto_label : ] : Auto Labeling Conf : {'method_name': 'selective', 'score_type': 'confidence', 'class_wise': 'independent', 'auto_label_err_threshold': 0.1, 'C_1': 0.0, 'ucb': 'sigma', 'fast': True}
[01/02/2024 08:05:41 PM : INFO  : auto_label : ] : Number of unlabeled points : 90000
[01/02/2024 08:05:41 PM : INFO  : auto_label : ] : Using number of validation points : 10000
[01/02/2024 08:05:41 PM : INFO  : auto_label : ] : Using Auto-Labeling Error Threshold = 0.1


[01/02/2024 08:05:41 PM : INFO  : auto_label : ] : Determining Thresholds : Class Wise : independent
[01/02/2024 08:05:41 PM : DEBUG : threshold_ : ] : MAX score = 0.9956614375114441, MIN score = 0.021912651136517525, delta = 9.737488289829344e-05
[01/02/2024 08:05:41 PM : DEBUG : threshold_ : ] : C_1 = 0.0 UCB = sigma
[01/02/2024 08:05:41 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 0   
[01/02/2024 08:05:41 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 1   
[01/02/2024 08:05:42 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=0.021912651136517525 for class 2   
[01/02/2024 08:05:42 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=0.2568794023245573 for class 3   
[01/02/2024 08:05:42 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 4   
[01/02/2024 08:05:42 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 5   
[01/02/2024 08:05:42 PM : INFO  : threshold_ : ] : auto-lab

{'auto_labeled_acc': 0.8813318689560955, 'coverage_1': 0.37331}

In [5]:
conf['auto_lbl_conf']['C_1'] = 0.25
conf['auto_lbl_conf']['auto_label_err_threshold']=0.10
dm.meta_df = meta_df_cp
meta_df_cp = copy.deepcopy(dm.meta_df)
auto_labeler = AutoLabeling(conf,dm,pl.cur_clf,logger)
out = auto_labeler.run()
dm.get_auto_labeling_counts()

[01/02/2024 08:06:11 PM : INFO  : auto_label : ] : ========================= Begin Auto-Labeling selective ==========================
[01/02/2024 08:06:11 PM : DEBUG : auto_label : ] : Auto Labeling Conf : {'method_name': 'selective', 'score_type': 'confidence', 'class_wise': 'independent', 'auto_label_err_threshold': 0.1, 'C_1': 0.25, 'ucb': 'sigma', 'fast': True}
[01/02/2024 08:06:11 PM : INFO  : auto_label : ] : Number of unlabeled points : 90000
[01/02/2024 08:06:11 PM : INFO  : auto_label : ] : Using number of validation points : 10000
[01/02/2024 08:06:11 PM : INFO  : auto_label : ] : Using Auto-Labeling Error Threshold = 0.1


[01/02/2024 08:06:12 PM : INFO  : auto_label : ] : Determining Thresholds : Class Wise : independent
[01/02/2024 08:06:12 PM : DEBUG : threshold_ : ] : MAX score = 0.9956614375114441, MIN score = 0.021912651136517525, delta = 9.737488289829344e-05
[01/02/2024 08:06:12 PM : DEBUG : threshold_ : ] : C_1 = 0.25 UCB = sigma
[01/02/2024 08:06:12 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 0   
[01/02/2024 08:06:12 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 1   
[01/02/2024 08:06:12 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=0.08277225308120251 for class 2   
[01/02/2024 08:06:12 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=0.41550386883318424 for class 3   
[01/02/2024 08:06:12 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 4   
[01/02/2024 08:06:12 PM : INFO  : threshold_ : ] : auto-labeling threshold t_i=inf for class 5   
[01/02/2024 08:06:12 PM : INFO  : threshold_ : ] : auto-la

{'auto_labeled_acc': 0.9310710498409331, 'coverage_1': 0.26404}